# Lab 8: 스테이지와 반정형 데이터

## Snowflake 스테이지 오브젝트 생성

👉 이 수업에서는 Education Services 팀에서 사용하도록 제공한 **S3**(Amazon Simple Storage Service) 버킷을 가르키는 스테이지를 생성할 것입니다.

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
your_db = user + '_DB'
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

### 스테이지 생성 🥋

1. Snowsight 오브젝트 브라우저에서 이전 실습에서 생성한 **(animal)_UTIL_DB** 데이터베이스를 선택합니다.
1. 그런 다음 **PUBLIC**이라는 이름의 스키마를 선택합니다.
1. 오른쪽 상단의 파란색 **Create** 버튼을 클릭하세요.
1. **Stage** > **External Stage** > **Amazon S3**를 선택하세요.

![Create stage option (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_create_stage_new_1.png)

스테이지 생성 대화 상자가 나타납니다.

![Stage creation dialog (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_create_stage_dialog_1.png)

1. **Stage Name** 텍스트 상자에 다음을 입력합니다: `like_a_window_into_an_s3_bucket`
1. **URL** 텍스트 상자에 다음 이름을 입력합니다: `s3://uni-lab-files`
1. **Directory table** 옵션이 선택된 상태로 유지되었는지 확인합니다.
1. 오른쪽 하단의 파란색 **Create** 버튼을 클릭합니다.
1. 다음 화면에서 사용할 Snowflake 가상 웨어하우스의 이름을 요청할 수도 있습니다. 이름이 지정된 **(animal)_WH** 웨어하우스를 선택하세요.

### 오브젝트 브라우저가 새로 생성한 스테이지의 사용 가능한 파일을 보여줍니다. 📓

![Stage files (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_new_stage_files.png)


💡 **팁**: 이 파일들과 폴더들은 Snowflake Education Services 팀이 소유하고 관리하는 AWS S3 버킷에 저장되어 있습니다. 이 과정을 위해 준비한 것입니다. 스테이지를 생성했기 때문에 해당 버킷의 파일 목록을 확인할 수 있습니다. 이는 해당 버키에 있는 파일들을 확인하고 액세스 할 수 있도록 하는 창과 같습니다. 이 버킷은 공개(Public)되어 있지만, 회사가 생성하는 버킷은 일반적으로 자격 증명이 필요할 것입니다.

### 스테이지인가, 스테이지가 아닌가? 📓 

Snowflake의 스테이지에 대해 혼란스럽고 이상하게 느껴질 수 있는 점 중 하나는, 방금 생성한 스테이지 오브젝트가 실제 위치가 아니라는 점입니다. 그 위치(파일들을 보유한 S3 버킷)는 이미 존재하고 있었습니다. 그렇다면, 방금 생성한 것은 무엇일까요?

사실, 우리가 생성한 것은 이미 파일이 준비된 특정 위치에 대한 정보를 Snowflake에 알려주는 오브젝트인 것입니다. 실제 스테이지 위치를 생성한 것이 아니라, 그 스테이지 위치를 들여다볼 수 있는 창과 같은 것을 만든 셈입니다. Snowflake 스테이지 오브젝트는 File format이 파일을 로드하는 작업을 쉽게 만들어주는 구성 정보를 보유하고 있다는 점에서 File Format과 비슷한 역할을 합니다.

때로는 Snowflake 스테이지를 정의할 때 접근 자격 증명도 제공해야 하지만, 이번 경우에는 제공하지 않았습니다. 우리가 생성한 스테이지는 단지 이름을 붙인 오브젝트로, 이미 파일이 준비된 S3 버킷을 가리키는 역할을 합니다. 

## SQL 셀에서 LIST 명령 사용하기 🥋 

### 새 스테이지의 파일을 보기 위해 LIST 명령 사용 🥋 

`LIST` 명령은 Snowflake 스테이지에 준비된 파일(로컬 파일 시스템에서 업로드되었거나 테이블에서 언로드된 파일)의 목록을 반환합니다.
- 이 명령은 `LS`로 줄여서 사용할 수도 있습니다.
- Snowflake 스테이지 오브젝트를 참조할 때는 이름 앞에 앰퍼샌드(`@`) 문자를 붙여야 합니다.

아래와 같은 `LIST` 명령을 직접 실행해 보세요.

In [ ]:
USE SCHEMA {{user}}_UTIL_DB.PUBLIC;

LIST @like_a_window_into_an_s3_bucket;

### Snowflake 오브젝트 명명 규칙 📓

Snowflake는 대소문자를 구분하지 않는다는 점에 주목하세요. 우리는 새 스테이지 오브젝트의 이름을 소문자로 입력했지만, Snowflake는 항상 모든 것을 **대문자**로 입력한다고 간주하고 자동으로 변환합니다. 따라서 오브젝트를 생성하거나 쿼리할 때 소문자 또는 혼합된 대소문자를 입력해도 Snowflake가 백그라운드에서 이를 대문자로 변환합니다.

(단, 생성 시 따옴표를 사용하면 그 이후로는 항상 따옴표를 사용해야 하며, 해당 따옴표 포함 이름을 사용해야 합니다.)

따라서 스테이지 오브젝트에 명령을 실행할 때는 어떤 대소문자 철자도 작동합니다. 그러나 S3는 매우 까다롭기 때문에 스테이지 오브젝트 이름을 넘어서 작업할 때는 매우 신중해야 합니다. 정확한 철자(파일 확장자를 포함한 대소문자 구분)를 사용해야 합니다.

![Stage files (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_list_command_2.png)

### Check 8 (OB08) 🔎

- `like_a_window_into_an_s3_bucket`이름으로 생성된 External Stage가 있나요?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB08', '{{user}}');

## `COPY INTO` 구문을 사용하여 데이터 로드 🥋 

### 토양 유형(Soil Type)을 위한 테이블 생성 🥋 

**(animal)_GARDEN_PLANTS** 데이터베이스의 **VEGGIES** 스키마에 생성하는지 확인하세요. 이를 위해 아래 SQL 셀에서 2번째 라인을 수정하여 해시 문자 `('#')`를 올바른 스키마 이름으로 대체한 수 셀을 실행하세요.

In [ ]:
-- replace the hash characters ('#') on the next line
USE SCHEMA {{user}}_GARDEN_PLANTS.VEGGIES;

CREATE OR REPLACE TABLE vegetable_details_soil_type
( plant_name VARCHAR(25)
 ,soil_type NUMBER(1,0)
);

### S3 버킷에서 새 테이블로 파일 로드하기 📓

이전에 Snowsight **데이터 로드** 화면을 사용하여 준비된 파일에서 테이블로 데이터를 복사한 적이 있습니다. 이 "마법사 기반(wizard-driven)" 접근 방식을 사용할 때 Snowflake는 이 작업을 수행하기 위해 코드 생성 및 실행을 백그라운드에서 처리합니다. 이번 데이터 로드에서는 프로그래밍 방식 접근법을 살펴볼 것입니다.

노트북의 SQL 셀에서 `COPY INTO` 구문을 사용할 것입니다.

`COPY INTO` 구문을 사용하려면, 다음 4 가지가 준비되어 있는 것이 가장 좋습니다:

1. 테이블

1. 스테이지 오브젝트

1. 파일

1. File Format (선택 사항)

**File Format**은 선택 사항입니다. 대안이 존재하지만, File Format을 사용하면 더 깔끔한 프로세스가 됩니다. 앞서 언급했듯이, File Format은 Snowflake가 스테이지에서 로드되는 데이터를 처리하는 방법에 대한 지침을 제공하는 오브젝트입니다. 아래 예제에서는 File Format을 정의하지 않고, 이러한 "지침"을 인라인으로 제공합니다.

### 실행할 수 있는 `Copy Into` 구문 🥋 

In [ ]:
COPY INTO vegetable_details_soil_type
FROM @{{user}}_util_db.public.like_a_window_into_an_s3_bucket
FILES = ( 'VEG_NAME_TO_SOIL_TYPE_PIPE.txt')
FILE_FORMAT = (
    TYPE=csv
    FIELD_DELIMITER = '|'
    SKIP_HEADER=1
);

### Check 9 (OB09) 🔎

- **(animal)_garden_plants.veggies** 스키마의 **vegetable_details_soil_type** 테이블에 42개의 행이 로드되었습니까?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB09', '{{user}}');

## 데이터 로드 팁과 요령 📓

- 모든 평면 파일(flat file)은 CSV(Comma Separated Values) 유형의 File Format을 사용하여 로드됩니다. 따라서 모든 평면 파일(TSV, Pipe Delimited, .txt 등)에 대해 `TYPE = CSV`를 사용하세요.

- **FIELD_DELIMITER** 속성은 매우 중요합니다. 이는 파일에서 사용되는 실제 컬럼 구분자와 일치해야 합니다.

- Data Load Wizard는 File Format(named 또는 inline)을 작성하는 데 도움을 줄 수 있습니다. 드롭 목록에서 필요한 설정을 선택한 다음 Show SQL 링크를 클릭하여 해당 설정에 대한 SQL 코드를 확인하세요.

## 챌린지 실습: 토양 유형(Soil Type) 조회 테이블 생성 🎯 

이 챌린지 실습에서는 **lu_soil_type**이라는 새 테이블을 생성하고 제공된 파일에서 데이터를 로드합니다. 아래 두 가지 방법 중 하나를 선택하여 진행할 수 있습니다:
- **Data Load** wizard 사용 **또는**
- 직접 `COPY INTO` 구문을 작성하고 실행 

:warning: 이 과제에서는 개략적인 지침만을 제공하며, 이전에 다룬 내용을 기반으로 스스로 "단계를 해결"해야 합니다. :warning:

먼저 테이블을 생성하세요. 테이블은 반드시 **(animal)_GARDEN_PLANTS** 데이터베이스의 **VEGGIES** 스키마에 생성해야 합니다.

In [ ]:
USE SCHEMA {{user}}_GARDEN_PLANTS.VEGGIES;

CREATE OR REPLACE TABLE lu_soil_type(
    soil_type_id STRING --NUMBER --,	
    --soil_type VARCHAR(15),
    --soil_description VARCHAR(75)
);

### 소스 데이터 파일 다운로드 🎯 

아래 Python 코드 셀을 실행하고 생성된 링크를 클릭하여 **LU_SOIL_TYPE.tsv** 파일을 다운로드합니다.

In [ ]:
snowpark_df = session.sql("SELECT GET_PRESIGNED_URL(@common_db.resources.course_files, 'LU_SOIL_TYPE.tsv')")
collected_data = snowpark_df.collect()
st.write('Click the following link to download the file:')
st.write(collected_data[0][0])

### 다운로드한 파일에서 테이블로 행 로드 🎯

**LU_SOIL_TYPE.tsv** 파일은 이전에 로드했던 **VEG_NAME_TO_SOIL_TYPE_PIPE.txt** 파일과 여러 File Format 속성을 공유하지만, 주요 차이가 하나 있습니다. 이것을 정리해보면:
- **TYPE**: 파일의 확장자는 **.tsv**입니다.(그래서 이것의 [유형](https://docs.snowflake.com/ko/sql-reference/sql/copy-into-table#type-csv)을 알아야 합니다.)
- **SKIP_HEADER**: 파일에는 헤더 행이 한 줄 있습니다.
- **FIELD_DELIMITER**: 파일은 파이프(`'|'`)로 구분된 것이 **아니라** **탭**으로 구분되어 있습니다. 탭은 다음 문자로 표현됩니다. `'\t'`

#### 옵션 1:
- Snowsight의 **Load Data** wizard를 사용.

#### 또는

#### 옵션 2:
- 위 내용을 참고하여 다음 SQL `COPY INTO` 구문을 수정하고 실행합니다.
- 대상 테이블의 이름(라인 1)을 입력합니다.
- File Format 옵션(라인 5-7)을 입력합니다.

In [ ]:
COPY INTO lu_soil_type
FROM @{{user}}_util_db.public.like_a_window_into_an_s3_bucket
FILES = ( 'LU_SOIL_TYPE.tsv')
FILE_FORMAT = (
    TYPE = CSV,
    SKIP_HEADER = 1
    FIELD_DELIMITER = '|',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
);

### Check 10 (OB10) 🔎

- **(animal)_garden_plants.veggies** 스키마의 **lu_soil_type** 테이블에 8개의 행이 로드되었습니까?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB10', '{{user}}');

## 반정형 데이터로 작업하기 📓

반정형 데이터는 전통적인 정형 데이터 표준을 따르지 않지만, 데이터 내의 개별적이고 명확한 엔티티를 식별하는 태그(라벨) 또는 기타 유형의 마크업을 포함하는 데이터를 말합니다. 반정형 데이터가 정형 데이터와 구별되는 두 가지 주요 속성은 중첩 데이터 구조와 고정된 스키마의 부재입니다:

- 반정형 데이터는 사전에 스키마 정의가 필요하지 않으며, 지속적으로 진화할 수 있습니다.(새로운 속성을 언제든지 추가할 수 있음)
- 정형 데이터가 데이터를 평면 테이블로 표현하는 것과 달리, 반정형 데이터는 중첩된 정모의 N-레벨 계층 구조를 포함할 수 있습니다.

다음은 Snowflake에서 지원하는 반정형 데이터 유형 중 하나인 JSON 데이터의 예입니다.

![Vegetable details table data (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_json_data_extract.png)

### `VARIANT` 데이터 유형

Snowflake는 반정형 데이터 저장을 지원하기 위해 [`VARIANT`](https://docs.snowflake.com/ko/sql-reference/data-types-semistructured#label-data-type-variant) 데이터 유형을 제공합니다. 이는 [`ARRAY`](https://docs.snowflake.com/ko/sql-reference/data-types-semistructured#array) 및 [`OBJECT`](https://docs.snowflake.com/ko/sql-reference/data-types-semistructured#object)(종종 반정형 데이터와 함께 사용됨)를 포함하여 다른 모든 데이터 유형의 값을 저장할 수 있는 데이터 유형입니다. 이 데이터 유형을 사용하면 Snowflake로 데이터를 로드할 때 반정형 데이터의 계층적/중첩된 형식을 유지할 수 있습니다.

### `VARIANT` 컬럼을 포함하는 테이블 생성 🥋

**(animal)_GARDEN_PLANTS** 데이터베이스의 **VEGGIES** 스키마에 **VEGETABLE_DETAILS_PLANT_HEIGHT** 테이블을 생성합니다. 이 테이블은 단일 컬럼을 포함하며, 컬럼의 데이터 유형은 `VARIANT`입니다.

In [ ]:
CREATE OR REPLACE TABLE {{user}}_GARDEN_PLANTS.VEGGIES.VEGETABLE_DETAILS_PLANT_HEIGHT (
	record VARIANT
);

### JSON 소스 데이터 파일 다운로드 🥋

아래 Python 코드 셀을 실행하고 생성된 링크를 클릭하여 JSON **veg_plant_height.json** 파일을 다운로드합니다.

브라우저에 따라 이 파일이 새 탭/창에서 직접 열리거나 다운로드될 수 있습니다. 다운로드된 경우, 로컬 시스템의 텍스트 편집기에서 파일을 열어 구조를 검토하세요.

💡 **팁**: 이 파일을 다운로드하는 이유는 파일의 구조와 내용을 검토하기 위함입니다. 데이터를 로드할 때는 이미 스테이지에 위치한 파일을 사용할 것입니다.

In [ ]:
snowpark_df = session.sql("SELECT GET_PRESIGNED_URL(@common_db.resources.course_files, 'veg_plant_height.json')")
collected_data = snowpark_df.collect()
st.write('Click the following link to download the file:')
st.write(collected_data[0][0])

### JSON 데이터를 로드하기 위한 `Copy Into` 구문 🥋 

이제 아래 구문을 실행하여 이 JSON 데이터를 새 테이블에 로드하세요.

- 정형 데이터가 아닌 반정형 데이터를 로드하기 위해 아래 **file format** 사양에서 무엇이 변경되었나요?

In [ ]:
COPY INTO vegetable_details_plant_height
FROM @common_db.resources.course_files/veg_plant_height.json
FILE_FORMAT = (TYPE = 'JSON');

### JSON 데이터를 로드한 테이블 쿼리 🥋

Snowflake는 `VARIANT`에 저장된 복잡한 계층적 데이터를 쿼리하기 위한 특별한 연산자와 함수를 제공합니다. 이를 곧 소개하겠지만, 지금은 정형 데이터 세트를 대상으로 사용하는 일반적인 SQL 구문을 사용하여 **vegetable_details_plant_height** 테이블에서 단일 행의 데이터를 검토해 보겠습니다.

In [ ]:
SELECT *
FROM vegetable_details_plant_height
LIMIT 1;

### 정형 데이터 쿼리하기

위의 쿼리에서 확인할 수 있듯이 **RECORD**라는 이름의 `VARIANT` 컬럼에는 네 개의 키-값 쌍이 포함되어 있습니다. 이는 데이터가 로드된 형식 그대로 유지되어 있어서 괜찮습니다. 하지만 엘레멘트를 개별적으로 처리하거나 보고서를 위해 정형된 형식으로 사용해야 할 때도 있을 것입니다.

다음은 Snowflake에서 반정형 데이터를 **탐색(traversing)**하는 개괄적인 지침입니다.
- `VARIANT` 컬럼 이름과 첫 번째 수준 엘레멘트 사이에 콜론 `:`을 넣습니다.
    - <column>:<level1_element>
- JSON 오브젝트의 계층적 경로에서 더 아래에 중첩된 엘레멘트에 접근하려면 점 표기법(dot notation)을 사용합니다.
    - <column>:<level1_element>.<level2_element>.<level3_element>
- **컬럼 이름**은 대소문자를 구분하지 않지만, 엘레멘트 이름들은 대소문자를 **구분**합니다.
- 엘레멘트 이름을 큰따옴표로 감싸는 것도 가능합니다.

다음 간단한 예제를 실행하여 **vegetable_details_plant_height** 테이블의 행을 **평면화(flatten)**(즉 정형 데이터로 표현)해 보세요.

In [ ]:
//Returns the data in a way that makes it look like a normalized table
SELECT 
    record:PLANT_NAME::STRING AS plant_name,
    record:UOM AS uom, -- no casting 
    record:LOW_END_OF_RANGE::INTEGER AS low_end_of_range,
    record:HIGH_END_OF_RANGE::INTEGER AS high_end_of_range
FROM vegetable_details_plant_height;

### `VARIANT` 값 캐스팅 📓

위 쿼리를 실행할 때 알아챘을 수도 있겠지만, **UOM** 컬럼 값이 다른 값들과 달리 **캐스팅**(다른 데이터 유형으로 변환)되지 않았다는 점입니다. 따라서 약간 다르게 보였으며, 다음과 같이 큰따옴표로 묶여 있었습니다: "F"

이는 해당 컬럼 값이 `VARCHAR` 또는 `STRING`(동의어)이라는 것을 의미하지 않습니다. 대신 이것은 여전히 `VARIANT` 값임을 나타냅니다. `VARIANT` 값은 문자열이 아니라 `VARIANT` 값이 문자열을 포함하고 있다는 것을 의미합니다.

Snowflake SQL에서는 다음과 같은 방법으로 데이터 유형을 캐스팅할 수 있습니다.
- [`CAST()`](https://docs.snowflake.com/ko/sql-reference/functions/cast) 함수 사용
- 대체 구문으로 `::` 연산자 사용 (예: **record:PLANT_NAME::STRING**)

## 챌린지 실습: 반정형 데이터를 보여주는 뷰 생성 🎯 

이 챌린지 실습에서는 새로운 [뷰(View)](https://docs.snowflake.com/ko/user-guide/views-introduction) 오브젝트를 생성할 것입니다. 뷰는 쿼리 결과를 테이블처럼 액세스할 수 있도록 합니다. 목표는 **vegetable_details_plant_height** 데이터를 정규화된 방식으로 표시하는 것입니다.

1. 아래 셀에 있는 템플릿 SQL을 수정하세요. 변경이 필요한 세 개의 라인은 `-- ***`로 표시되어 있습니다.
1. **UOM** 컬럼을 `VARCHAR`로 **캐스팅**합니다.
1. **LOW_END_OF_RANGE** 와 **HIGH_END_OF_RANGE** 컬럼의 순서를 **교체**하세요.

In [ ]:
-- modify the following code according to the instructions above, then run to create the object
CREATE OR REPLACE VIEW vegetable_details_plant_height_vw AS 
SELECT 
    record:PLANT_NAME::STRING AS plant_name,
    record:UOM::VARCHAR AS uom, -- *** 
    record:HIGH_END_OF_RANGE::INTEGER AS high_end_of_range, -- ***
    record:LOW_END_OF_RANGE::INTEGER AS low_end_of_range -- ***

FROM vegetable_details_plant_height;

In [ ]:
-- then run this query against your new view to confirm the output
SELECT *
FROM vegetable_details_plant_height_vw;

### Check 11 (OB11) 🔎

- **(animal)_GARDEN_PLANTS** 데이터베이스의 **VEGGIES** 스키마에 **vegetable_details_plant_height_vw**라는 뷰를 생성했습니까?
- **UOM**이 `VARCHAR`(텍스트) 컬럼으로 캐스팅되었으며, **HIGH_END_OF_RANGE** 컬럼이 세 번째 위치에, **LOW_END_OF_RANGE** 컬럼이 네 번째 위치에 있습니까?
- 작업을 확인하려면 채점용 Stored Procedure를 호출하세요.

In [ ]:
CALL common_db.resources.local_grader('OB11', '{{user}}');

## 지식 테스트 🔎

아래 Python 셀을 실행하여 Streamlit 기반의 위젯을 표시하고 Snowflake 특징과 기능에 대한 질문에 답하세요. 지금은 이 코드가 무엇을 하는지 이해할 필요가 없습니다. 그냥 코드를 실행하세요.

질문에 올바르게 답해야 다음 섹션으로 진행할 수 있습니다.

In [ ]:
st.divider()
question = "왜 우리가 Snowflake 스테이지 오브젝트에 LIKE_A_WINDOW_INTO_AN_S3_BUCKET이라는 이름을 부여했다고 생각하십니까?"
options = ["아래 보기 중 고르세요...",
           "A) 짧아서 입력하기 쉬우니까", 
           "B) 스테이지는 창문처럼 쉽게 깨질 수 있기 때문에", 
           "C) 스테이지는 위치처럼 들리며, 이 경우 위치는 우리가 만든 Snowflake 오브젝트가 아니라 S3 버킷이기 때문에"]                                 

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...": 
        ''
    else:
        answer = 'f61225ec7b84a3fdde9fe670dd382be4'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

In [ ]:
st.divider()
question = "대문자, 소문자, 혼합된 경우? 그게 중요한가요? 아래 설명 중 Snowflake 대소문자 구분에 대한 올바른 설명은 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) Snowflake는 항상 사용자가 대문자로 입력하려고 했다고 가정합니다. 단 큰따옴표로 묶인 경우는 예외입니다.", 
           "B) HAPPY라는 테이블을 생성하면, Snowflake는 실제로 이를 happy라는 이름으로 만듭니다.", 
           "C) \"hAppY\"라는 테이블을 생성하면, 이를 쿼리할 때마다 이름을 작은따옴표로 묶어야 합니다."]                                 

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...": 
        ''
    else:
        answer = 'daa61f51d82b7039bd0fd9faa61d23eb'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 다음 단계

실습 단계를 완료하고 **지식 테스트** 질문에 올바르게 답했다면, Snowflake 강사의 안내에 따라 다음 노트북으로 진행하세요.